In [1]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark.sql.functions import when
from pyspark.sql.types import StructType, StructField, ShortType, StringType
import datetime
import os

master = "spark://zy-ubuntu:7077"  
os.environ['PYSPARK_SUBMIT_ARGS'] = f'--master {master} --driver-memory 4g --total-executor-cores 8 --executor-memory 8g --packages org.postgresql:postgresql:42.1.1 pyspark-shell'

In [2]:
spark = SparkSession.builder \
    .appName("download_date") \
    .getOrCreate()


In [3]:
# create a df with year, month, day with today's date
now = datetime.datetime.now()

data = [(now.year,now.month,now.day)]

schema = StructType([ \
    StructField("year",ShortType(),True), \
    StructField("month",ShortType(),True), \
    StructField("day",ShortType(),True), \
  ])
 
dl_date_df = spark.createDataFrame(data=data,schema=schema)
dl_date_df.show()

+----+-----+---+
|year|month|day|
+----+-----+---+
|2021|    6| 13|
+----+-----+---+



In [4]:
# insert dataset download date into db
dl_date_df.write.format('jdbc').options(
      url='jdbc:postgresql://localhost:5433/imdb',
      driver='org.postgresql.Driver',
      dbtable='download_date',
      user='admin',
      password='password'
      ).mode('append').save()